# Document Clustering and Topic Modeling

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

怎么对文档进行分类，cluster，现在用的是 1gram，用2gram能传达更多信息。custome cluster
这个project要更好的话，可以再把Kmeans的值k调整上去，数据量可以调，stopwords还可以用别的数据包，n也可与调整。
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,#这些数字，都是hyperparamater，都需要自己去调的，都要自己看的
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))
                                 这里面的数据都可以调整。

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [1]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [2]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
# file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
# file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [1]:
import numpy as np #主要用
import pandas as pd #主要用
import nltk #主要用于做自然语言 处理
import matplotlib.pyplot as plt
import seaborn as sns

import gensim
# REGULAR EXPRESSION
import re#regular expression


from sklearn import decomposition #sklearn
from sklearn.feature_extraction.text import TfidfVectorizer 

nltk.download('punkt')
nltk.download('stopwords')#下载一些words

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /Users/qianyirou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qianyirou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', header=0, error_bad_lines=False) 
# 有一些数据不是很好，error_bad_lines用于数据探索，tsv：tab分隔，两个或者四个空格那么宽

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960204 non-null  object
 1   customer_id        960204 non-null  int64 
 2   review_id          960204 non-null  object
 3   product_id         960204 non-null  object
 4   product_parent     960204 non-null  int64 
 5   product_title      960202 non-null  object
 6   product_category   960204 non-null  object
 7   star_rating        960204 non-null  int64 
 8   helpful_votes      960204 non-null  int64 
 9   total_votes        960204 non-null  int64 
 10  vine               960204 non-null  object
 11  verified_purchase  960204 non-null  object
 12  review_headline    960197 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960200 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [4]:
# Remove missing value
# df = df.dropna(subset=['review_body'])
text = df['review_body'].dropna()
text

0         Absolutely love this watch! Get compliments al...
1              I love this watch it keeps time wonderfully.
2                                                 Scratches
3         It works well on me. However, I found cheaper ...
4         Beautiful watch face.  The band looks nice all...
                                ...                        
960199    It's a great watch - but you don't have a chan...
960200    Well worth the price. This is a light weight, ...
960201    Dear Targeteers,<BR>This watch is exelent. it ...
960202    In the old days, the common hearing battery in...
960203    I have found that a #393 watch battery is the ...
Name: review_body, Length: 960056, dtype: object

In [5]:
text.isnull().sum()

0

In [6]:
data = text.loc[:10000]

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [7]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 179 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string 
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word not in stopwords] #每一句里面找每个词

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens: #找只有字母的东西，不要数字，纯字母
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# tokenization without stemming
def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [9]:
# tokenization and stemming
tokenization_and_stemming(data[0]) #i是问题，需要调整

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'i',
 'wear',
 'dainti']

Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [10]:
# 1. do tokenization and stemming for all the documents
# 2. also just do tokenization for all the documents
# the goal is to create a mapping from stemmed words to original tokenized words for result interpretation.
docs_stemmed = []
docs_tokenized = []
for i in data:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

In [11]:
# create a mapping from stemmed words to original words
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency

***example:***

document1: "Arthur da Jason"

document 2: "Jason da da huang"

document1: tf-idf [1, 0.5, 0.5, 0];  document2: tf-idf [0, 1, 0.5, 1]  

2-gram: 

document 1: Arthur da, da Jason; document 2: Jason da, da da, da huang bigram

3-gram:

document 1: Athur da Jason;  document 2: Jason da da, da da huang

[Arhur, da, Jason...]

In [18]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,#这些数字，都是hyperparamater，都需要自己去调的，都要自己看的
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))
#tokenizer=tokenization_and_stemming：pass了一个function
tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 9993 reviews and 230 terms.


In [19]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming(text)>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [20]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [53]:
# print out words
tf_selected_words

["'m",
 "'s",
 'abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'appear',
 'arriv',
 'attract',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'br',
 'bracelet',
 'brand',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clear',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'coupl',
 'crystal',
 'cute',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'design',
 'dial',
 'differ',
 'difficult',
 'digit',
 'disappoint',
 'display',
 'durabl',
 'easi',
 'easili',
 'eleg',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'figur',
 'fine',
 'fit',
 'function',
 'gave',
 'gift',
 'glass',
 'goe',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',

# Calculate Document Similarity

In [54]:
# use cosine similarity to check the similarity for two documents
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_matrix)#两个向量距离可以用cos表示
cos_matrix

array([[1.        , 0.42293542, 0.        , ..., 0.        , 0.03411147,
        0.03158665],
       [0.42293542, 1.        , 0.        , ..., 0.        , 0.0806541 ,
        0.07468433],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.03411147, 0.0806541 , 0.        , ..., 0.        , 1.        ,
        0.04111185],
       [0.03158665, 0.07468433, 0.        , ..., 0.        , 0.04111185,
        1.        ]])

# Part 4: K-means clustering

In [23]:
# k-means clustering，这里只有1-gram
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)#创建km的object，然后fit，这就是一个model
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [24]:
clusters#每一个object属于一个cluster

[3,
 2,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 1,
 3,
 4,
 3,
 3,
 0,
 3,
 3,
 3,
 2,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 2,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 0,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 4,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 4,
 3,
 3,
 3,
 0,
 3,
 0,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 0,
 4,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 0,
 4,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 3,
 4,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 4,
 3,
 3,
 3,
 3,
 0,
 3,
 3,
 3,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 0,
 2,
 2,
 1,
 3,
 0,
 0,
 3,
 0,
 3,
 1,
 3,
 3,
 4,
 3,
 3,
 1,
 3,
 1,
 3,
 1,
 2,
 0,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 0,
 3,
 3,
 3,
 3,
 4,
 3,
 2,
 0,
 3,
 0,
 1,
 3,
 3,
 3,
 3,
 3,
 1,
 0,
 1,
 3,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 1,
 3,
 0,
 3,
 4,
 3,
 3,
 1,
 0,
 3,
 4,
 3,
 1,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 1,
 3,
 3,


In [25]:
tfidf_matrix#存成一个matrix，节省空间，这个 就是 输入

<9993x230 sparse matrix of type '<class 'numpy.float64'>'
	with 76888 stored elements in Compressed Sparse Row format>

## 4.1. Analyze K-means Result

In [35]:
# create DataFrame films from all of the input files.创建dataframe，把review跟cluster对应起来
product = { 'review': df[:len(clusters)].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [36]:
frame

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",3
1,Kenneth Cole New York Women's KC4944 Automatic...,2
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,3
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,3
4,Orient ER27009B Men's Symphony Automatic Stain...,3
...,...,...
9988,HTS 194A0 5 Pc Watch Band Link Remover Tool Kit,3
9989,Sannysis New Women Knitted Braided Weaved Rope...,4
9990,Mokingtop Fashion Watch for Ladies Dress Quart...,3
9991,Timex Grand Street Watch,3


In [38]:
print ("Number of reviews included in each cluster:")#每个cluster里面有多少个数据
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
3,6897
4,1112
2,780
0,628
1,576


In [39]:
km.cluster_centers_ 
#这里是算中心点的向量的tf-idf值，一堆东西的中心点，每一次中心点是1*245的vector，只是里面的值不一样，里面的值代表了对应feature的tfidf
#聚类之后，每个cloumn代表一个feature，一个word，每一个维度代表一个feature，每一个feature代表一个值，word的tf-idf，每个向量是一个cluster center，，这下面的每一个值都是一个feature，每一个值都对应一个word
#[0.01353968, 0.04949882, 0.00533351, ..., 0.00635103, 0.01652616, 0.01261386]是245个词

array([[0.00144084, 0.01509007, 0.        , ..., 0.00326701, 0.00243774,
        0.00088392],
       [0.00185939, 0.01637578, 0.        , ..., 0.00302717, 0.00666087,
        0.        ],
       [0.00537626, 0.02134908, 0.0009837 , ..., 0.001514  , 0.00535325,
        0.00564995],
       [0.01512794, 0.04717616, 0.00518897, ..., 0.0087608 , 0.0236437 ,
        0.01568189],
       [0.00435704, 0.0240456 , 0.0013176 , ..., 0.00491201, 0.00900906,
        0.01238181]])

In [41]:
km.cluster_centers_.shape #230个feature，就是230个词

(5, 230)

In [43]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] #argsort:选出每个向量里面值最大的，然后排序
#把每一个向量里面的东西排序，并且返回tfidf最大值所在的index
#每个值都是一个tfidf，一个中心点就是表示一个tfidf，我们找的是一堆230个词里面index对应的词


Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
    #把每一个里面的 前面6个对应的word都拿了出来 ，打印出来，word的tf-idf，类似于权重
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()
    #clustering有时候需要硬看

<Document clustering result by K-means>
Cluster 0 words:good,product,watch,looks,quality,price,
Cluster 0 reviews (628 reviews): 
Luminox Men's 3081 Evo Navy SEAL Chronograph Watch, Casio G-shock Mudman Multiband6 Japanese Model [ Gw-9300-1jf ], Voguestrap TX046801XL Allstrap 16-20mm Brown Extra-Long-Length Fits Fast-Wrap Expedition Watchband, XOXO Women's XO110 Silver Dial Gold-tone Bracelet Watch, Akribos XXIV Men's AK787YGBU Quartz Movement Watch with Blue Dial and Yellow Gold Stainless Steel Bracelet, AMPM24 Men's Hand-winding Mechanical Watch Black Leather Watchband Skeleton PMW069, Gotham Men's Silver-Tone Ultra Thin Railroad Open Face Quartz Pocket Watch # GWC15022S, Casio Unisex MRW200H-2BV Neo-Display Black Watch with Resin Band, Unisex Fashion Sport Watch Multifunction Multi-colour Led LAnalog Digital Waterproof Alarm Wristwatch, Seiko Wall Clock Silver-Tone Metallic Case Luminous  Numerals, Casio Men's STB-1000-1CF OmniSync Sports Gear Bluetooth Fitness Smartwatch, Casio Men

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [44]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values，以为LDA要求输入一定是整数，不能小数，所以用CountVectorizer
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))
#fit_transform：训练一下，并且transform成tfidf
tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In total, there are 9993 reviews and 230 terms.


In [46]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)
#(10000, 5)：10000个document，5个term，10000行，5个topic，每个 doc属于某个cluster的概率，每一行加起来一定是1.

(9993, 5)
[[0.02515446 0.02570174 0.55268951 0.37110568 0.02534862]
 [0.05051359 0.05081081 0.47081939 0.37679618 0.05106004]
 [0.10033114 0.10098614 0.10000023 0.5986823  0.10000019]
 ...
 [0.06743446 0.06749271 0.0669912  0.73085639 0.06722524]
 [0.04037585 0.64031201 0.04069351 0.23825675 0.04036189]
 [0.03457308 0.03372411 0.86349361 0.0337688  0.03444039]]


In [47]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 230)
[[5.15063675e+01 4.84543390e+02 1.04625741e+01 ... 7.91142600e+01
  1.46391271e+02 3.72445225e-01]
 [4.80822591e+01 7.43522621e+01 1.05863365e+00 ... 1.34840463e+02
  2.07405994e-01 2.73996467e+02]
 [7.93325097e+01 4.72438316e+02 7.57247451e+00 ... 6.06615214e-01
  2.65424491e+02 6.21737396e-01]
 [3.15061805e+02 7.88802130e+02 1.43759054e+02 ... 1.05157919e+01
  5.71462514e+01 2.92792274e+02]
 [6.70170585e+01 3.92863902e+02 2.14726400e+00 ... 9.22869685e-01
  4.31830580e+02 2.17075856e-01]]


In [48]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.03,0.03,0.55,0.37,0.03,2
Doc1,0.05,0.05,0.47,0.38,0.05,2
Doc2,0.10,0.10,0.10,0.60,0.10,3
Doc3,0.03,0.59,0.03,0.03,0.31,1
Doc4,0.36,0.22,0.14,0.18,0.10,0
Doc5,0.34,0.03,0.26,0.34,0.03,0
Doc6,0.03,0.03,0.88,0.03,0.03,2
Doc7,0.47,0.45,0.03,0.03,0.03,0
Doc8,0.01,0.01,0.38,0.40,0.20,3
Doc9,0.58,0.02,0.02,0.36,0.02,0


In [49]:
df_document_topic['topic'].value_counts().to_frame()#让topic相对均匀

,topic
2,2234
1,2161
0,2043
4,2032
3,1523


In [50]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()
#每一个topic里面词的权重 

[[5.15063675e+01 4.84543390e+02 1.04625741e+01 ... 7.91142600e+01
  1.46391271e+02 3.72445225e-01]
 [4.80822591e+01 7.43522621e+01 1.05863365e+00 ... 1.34840463e+02
  2.07405994e-01 2.73996467e+02]
 [7.93325097e+01 4.72438316e+02 7.57247451e+00 ... 6.06615214e-01
  2.65424491e+02 6.21737396e-01]
 [3.15061805e+02 7.88802130e+02 1.43759054e+02 ... 1.05157919e+01
  5.71462514e+01 2.92792274e+02]
 [6.70170585e+01 3.92863902e+02 2.14726400e+00 ... 9.22869685e-01
  4.31830580e+02 2.17075856e-01]]


,'m,'s,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,...,week,weight,white,wife,wish,work,worn,worth,wrist,year
Topic0,51.506367,484.543390,10.462574,5.213193,27.747213,96.268676,79.662840,0.201370,20.401856,10.888772,...,7.207897,27.314431,56.171236,0.203787,38.367435,41.290329,15.373403,79.114260,146.391271,0.372445
Topic1,48.082259,74.352262,1.058634,0.240220,7.649383,0.206695,0.202707,25.659934,57.312199,10.057081,...,239.599723,0.201993,0.273076,0.204020,1.098836,993.133670,37.133505,134.840463,0.207406,273.996467
Topic2,79.332510,472.438316,7.572475,136.052278,0.202785,5.317951,33.801602,0.201598,14.587916,22.313703,...,0.227742,2.704419,0.259739,133.765265,9.845686,1.264007,3.126039,0.606615,265.424491,0.621737
Topic3,315.061805,788.802130,143.759054,9.190588,93.431006,54.639945,109.743789,119.490927,23.267369,105.155679,...,154.452847,0.305834,15.572891,17.471330,66.736448,252.522336,75.796368,10.515792,57.146251,292.792274
Topic4,67.017058,392.863902,2.147264,0.303720,55.969614,42.566733,42.589062,15.446171,0.430660,22.584765,...,0.511792,124.473322,62.723058,0.355598,43.951595,66.789658,14.570685,0.922870,431.830580,0.217076


In [51]:
# print top n keywords for each topic：选前面几个词，更好看的打印出来，但 这里topic太少，review相对多 ，看不出结果。
#解决办法：增加LDA数字，成百个 更好
#LDA的结果，更具有代表性，数学推理更完备，但是LDA比kmeans慢
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,br,watch,band,look,great,qualiti,price,strap,'s,like,expect,good,cheap,leather,n't
Topic 1,watch,good,work,great,batteri,product,look,day,n't,price,buy,year,return,month,replac
Topic 2,love,watch,beauti,perfect,'s,great,look,gift,fit,bought,wrist,awesom,thank,husband,band
Topic 3,watch,time,n't,'s,use,second,set,hand,excel,day,ve,wear,need,like,'m
Topic 4,watch,nice,look,like,face,easi,read,wrist,realli,'s,time,small,great,light,big
